<a href="https://colab.research.google.com/github/Sam-Ny/PySpark-Project/blob/main/PySpark_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=aee34d99e29f24625bc410d3bb0272ca79995a706c0f841f7b19e5810128028f
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as func
from pyspark.sql.functions import *
from pyspark.sql.functions import regexp_replace

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! apt-get install git
! git clone https://github.com/Sam-Ny/PySpark-Project.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'PySpark-Project'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 4.79 KiB | 4.79 MiB/s, done.


**PySpark Project Start**

In [89]:
#Creating the SparkSession
spark = SparkSession.builder.appName("FirstSparkProject").getOrCreate()

In [90]:
df = spark.read.csv('/content/PySpark-Project/PySpark Code/googleplaystore.csv',sep=',',header='true',escape='"',inferSchema='true')

In [ ]:
df.count()

In [73]:
df.show(1,truncate=False)

+----------------------------------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|App                                           |Category      |Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres      |Last Updated   |Current Ver|Android Ver |
+----------------------------------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Candy Camera & Grid & ScrapBook|ART_AND_DESIGN|4.1   |159    |19M |10,000+ |Free|0    |Everyone      |Art & Design|January 7, 2018|1.0.0      |4.0.3 and up|
+----------------------------------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [74]:
df.printSchema() #Schema is not properly inferred because the data has some nulls.

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



# Data Cleaning





In [91]:
df = df.drop("size","Content Rating","Last Updated","Current Ver","Android Ver")

In [80]:
df.show(2,truncate=False)

+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|App                                           |Category      |Rating|Reviews|Installs|Type|Price|Genres                   |
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|Photo Editor & Candy Camera & Grid & ScrapBook|ART_AND_DESIGN|4.1   |159    |10,000+ |Free|0    |Art & Design             |
|Coloring book moana                           |ART_AND_DESIGN|3.9   |967    |500,000+|Free|0    |Art & Design;Pretend Play|
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
only showing top 2 rows



In [92]:
df2=df.withColumn('Reviews',col('Reviews').cast(IntegerType()))\
    .withColumn('Installs',regexp_replace(col("Installs"),"[,]",""))\
    .withColumn('Installs',regexp_replace(col("Installs"),"[^0-9]",""))\
    .withColumn('Installs',col('Installs').cast(IntegerType()))\
    .withColumn('Price',regexp_replace(col("Price"),"[$]",""))\
    .withColumn('Price',col('Price').cast(IntegerType()))

In [93]:
df2.show(2,truncate=False)
df2.dtypes

+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|App                                           |Category      |Rating|Reviews|Installs|Type|Price|Genres                   |
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|Photo Editor & Candy Camera & Grid & ScrapBook|ART_AND_DESIGN|4.1   |159    |10000   |Free|0    |Art & Design             |
|Coloring book moana                           |ART_AND_DESIGN|3.9   |967    |500000  |Free|0    |Art & Design;Pretend Play|
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
only showing top 2 rows



[('App', 'string'),
 ('Category', 'string'),
 ('Rating', 'double'),
 ('Reviews', 'int'),
 ('Installs', 'int'),
 ('Type', 'string'),
 ('Price', 'int'),
 ('Genres', 'string')]

# Data Analysis

Using Spark Sql and pyspark dataframe operations to do data analysis

In [100]:
df2.createOrReplaceTempView("apps")

In [ ]:
#Using Sql queries
df3=spark.sql("select app,category from apps")
df3.show(2,truncate=False )

#Using pyspark operations
df3=df2.select("category","app")
df3.show(2,truncate=False )

Top 10 Reviews given to the apps

In [ ]:
#Using Sql queries
query = '''select app,sum(reviews) as total_reviews from apps group by app order by 2 desc'''
df3 = spark.sql(query)
df3.show(10, truncate=False)

#Using pyspark operations
df3 = df2.groupBy('app')\
         .agg(sum('reviews').alias('total_reviews'))\
         .orderBy(sum('reviews').desc())
df3.show(10, truncate=False)

# Or we can also use order by as

df3 = df2.groupBy('app')\
         .agg(sum('reviews').alias('total_reviews'))\
         .orderBy(desc('total_reviews'))
df3.show(10, truncate=False)